# Fluid modeling

PorePy supports the introduction of multiphase, multicomponent fluids into its models.

This tutorial focuses on relevant mixin-methods and two approaches to modeling of fluid properties:

- Heuristic approach using functions in the form of constitutive laws.
- Equation of State (EoS) -based approach using the respective object for computations.

### Table of contents

1. [Basics and default behavior in PorePy](#basics-and-default-behavior-in-porepy)
2. [Modeling component and phase context](#modeling-component-and-phase-context)
3. [Modeling of fluid properties](#modeling-of-fluid-properties)
    - [Heuristic approach (default)](#heuristic-approach)
    - [Equation of State approach (advanced)](#equation-of-state-approach)
    - [Hybrid approach (advanced)](#hybrid-approach)
4. [Final remarks](#final-remarks)

## Basics and default behavior in PorePy

Fluids can be defined in terms of two contexts: components and phases.

While the number of components is something the modeler chooses naturally, the number (and more importantly type) of phases is
an unknown number in the general thermodynamic sense.
In PorePy, the modeler must define the (maximal) number and types of phases expected in the course of simulations.

The modeling can then be roughly performed in 3 steps:

1. Defining the components in the fluid (water, CO2, ...)
2. Defining the expected phases (1 gas phase, 2 liquid phases for example)
3. Introducing a thermodynamic model for fluid properties (densities, enthalpies, ...)

> **Note on distinction between fluid, phase and component properties:**
> 
> It makes no sense to talk about the density of a component, say water.
> Liquid and gaseous water have greatly different densities.
> I.e., extensive properties like density and enthalpy are always related to the phase.
> Multiple components can be present in a phase and influence the properties.
> Extensive properties of fluid mixtures (consisting of multiple components in multiple phases), depend in a clearly defined manner on the respective properties of present phases (usually a weighted sum of phase properties).

The default fluid in PorePy models consists of 1 component, and 1 phase containing that component.
In this simple case, the fluid properties are equal to the (single) phase properties.

The class [FluidMixin](../src/porepy/compositional/compositional_mixins.py#L1042) provides respective functionalities for creating a fluid, and is part of all models in PorePy.

Legacy implementations provide a simple customization of the single-component without major effort:


In [1]:
import porepy as pp
from porepy.applications.material_values.fluid_values import water

model_params = {
    'material_constants': {
        'fluid': pp.FluidComponent(**water)
    }
}

model = pp.SinglePhaseFlow(model_params)
model.prepare_simulation()

Above code creates the default single-phase, single-component fluid and is ready for simulations right away.
It is compatible with all constitutive laws for single-phase flow found in the [fluid property library](../src/porepy/models/fluid_property_library.py).

In [2]:
print(model.fluid.num_components)
print(model.fluid.num_phases)
print(model.fluid.reference_component.name)

1
1
water


Note however, that the `FluidMixin` is part of all models found in the `models` subpackage.
It comes with some general implementations of fluid properties, which are covered in a later section.
To override those general implementations, constitutive laws must be treated as pure mixins (as intended).
These constitutive laws then represent the thermodynamic model of the fluid.

For customization, they must be on top in the MRO:

In [3]:
class CustomSinglePhaseFlow(
    pp.constitutive_laws.FluidDensityFromPressureAndTemperature,
    pp.SinglePhaseFlow,
):
    """Just for demonstration purpose. Note that the base model has no notion of
    temperature."""
    

## Modeling component and phase context

For customizing the component context, the method we need to override is `get_components`.
For customizing the phase context, we need to override `get_phase_configuration`.

Below example `MyFluid` will model a 2-component, 3-phase fluid, assuming that the simulation
is in pressure and temperature ranges where water and CO2 can separate into two immiscible, liquid-like phases.

In [4]:
from typing import Sequence


class MyFluid(pp.PorePyModel):

    def get_components(self) -> Sequence[pp.FluidComponent]:
        return [
            # Some custom viscosity values for a later section.
            pp.FluidComponent(name='H2O', viscosity=1e-3),
            pp.FluidComponent(name='CO2', viscosity=1e-4),
        ]
    
    def get_phase_configuration(
        self, components: Sequence[pp.compositional.ComponentLike]
    ) -> Sequence[tuple[pp.compositional.PhysicalState, str]]:
        # Phases are configured by defining the physical state and a name.
        return [
            (pp.compositional.PhysicalState.liquid, 'aqueous'),
            (pp.compositional.PhysicalState.liquid, 'non_aqueous'),
            (pp.compositional.PhysicalState.gas, 'gaseous'),
        ]

Each component has an associated overall fraction variable accessible as `component.fraction`.
Each phase has an associated fraction variable *and* a saturation variable, accessible as
`phase.fraction` and `phase.saturation` respectively. The fractions can be either massic or molar and depend on the
implementation of the fluid properties. Saturations are always volumetric fractions.

By default, the first, returned component and phase (configuration) are set as the reference component and phase respectively.
This choice has an impact on both variables and equations created, depending on the model parametrization.

In [5]:
model_params = {
    'eliminate_reference_phase': True,
    'eliminate_reference_component': True,
}

Both parameters are by default `True`. If the reference phase is eliminated, its fraction and saturation
are not introduced as independent variables, but expressed through unity:

$$
\varphi_{r} = 1 - \sum_{i\neq r} \varphi_i
$$

Analogously, if the reference component is eliminated, its overall fraction is expressed through unity.
Additionally, if the reference component is eliminated, compositional flow models do not introduce a mass balance equation
for the respective component.

We note these effects here but refer to the code in `porepy.models.compositional_flow` for more details.

Now that we know how to introduce phases and components into the model, we can talk about the interplay
between the two contexts: Components in a phase.

In general, a component in a phase is associated with an additional degree of freedom, the partial fraction `phase.partial_fraction_of[component]`

The behavior of which component is present in which phase is determined by the method `set_components_in_phases`.
Its default behavior introduces every component in every phase (unified assumption).
For fluids, this assumption is realistic since traces of every component can be found in every phase, if that phase is present in the thermodynamic sense (phase appearance/disappearance).
Future work will encompass solid components and solid phases, and expecting a fluid component like water inside a solid phase with granite or sand is nonsense. We can avoid introducing respective DoFs.

To describe that logic, let's assume a special model based on `MyFluid` which we simplify such that each liquid phase contains only an associated component.

In [6]:
class MySpecialFluid(MyFluid):

    def set_components_in_phases(
        self, components: Sequence[pp.Component], phases: Sequence[pp.Phase]
    ) -> None:
        h2o, co2 = components
        aqu, non_aqu, gas = phases

        # Aqueous phase contains only water, the non-aqueous only CO2.
        # The gas phase contains both, since gases always mix.
        aqu.components = [h2o]
        non_aqu.components = [co2]
        gas.components = [h2o, co2]

With above mixin `MySpecialFluid`, the two liquid phases will contain only 1 component.
This means that only 1 partial fraction per liquid phase can exist. If only 1 can exist, it is by default the scalar 1.
The gas phase has two partial fractions associated, one for each component. Since the unity constraint holds analogously for partial fractions, the partial fraction of water in the gas phase is eliminated **always**. This is contrary to the phase and overall fractions, where the choice of number of degrees of freedom is relevant to treat the issue of
phase appearance and disappearance.

To summarize:

With default model parameters, `MySpecialFluid` will introduce in total 6 degrees of freedom in each subdomain cell:

- 1 overall fraction of CO2
- 2 phase fractions for the non-aqueous and the gas phase (under certain conditions, see later section)
- 2 phase saturations for the non-aqueous and gas phase
- 1 partial fraction for CO2 in the gas phase.

This behavior, including some utility functions, is implemented in the [MixtureDOFHandler](../src/porepy/compositional/compositional_mixins.py#106), which is inherited by the
[CompositionalVariablesMixin](../src/porepy/compositional/compositional_mixins.py#536).
The latter must be part of any PorePy model with more than 1 phase or component.

> **Note on phase fractions:**
>
> Phase fractions are quantities which do not usually appear in flow & transport problems, but in phase equilibrium or chemical problems.
> Hence it is not created as a DOF by default, but its creation must be triggered by defining an equilibrium type in the `model_params['equilibrium_type']= 'some_string'`.
>
> They can appear under non-isothermal conditions though, since the specific enthalpy and specific internal energy of the fluid mixture are sums of phase enthalpies/energies weighed with phase fractions.
> With those more complex models, the user must be well aware of the DOFs required and how to create them, and how the (specific) energies are implemented.

> **Note on extended partial fractions:**
>
> PorePy supports formulations of the unified equilibrium problem. It is triggered by including the keyword `'unified'` in `model_params['equilibrium_type']='unified-some-other-keywords'`.
> The unified formulation represents a persistent set of variables, including the so-called extended partial fractions, which are variables also for vanished phases. They are always independent quantities, even in phases with only 1 component.
> They are accessed by `phase.extended_fraction_of[component]`.
>
> In the unified setting, the (physical) partial fractions are always dependent quantities obtained by normalization of extended fractions.


## Modeling of Fluid Properties

Fluid properties, like all other terms entering some equation, are implemented as callables.
They take a sequence of grids (subdomains or boundary grids) and return an AD operator.

Since PorePy allows for a dynamic modeling of phases, these callables must also be handled dynamically.
We achieve this by implementing various "function factories".

### Heuristic approach

The heuristic approach assumes that the fluid property is some analytical expression which can be handled by PorePy's AD framework.

Let's consider the a heuristic law for density of form

$$
\rho(p) = \rho_0 e^{c(p - p_0)}
$$

This density function is already available in the constitutive laws in PorePy and is implemented as follows:

In [7]:
import inspect

print(inspect.getsource(pp.constitutive_laws.FluidDensityFromPressure.density_of_phase))

    def density_of_phase(self, phase: pp.Phase) -> ExtendedDomainFunctionType:
        """Mixin method for :class:`~porepy.compositional.compositional_mixins.
        FluidMixin` to provide a density exponential law for the fluid's phase.

        .. math::
            \\rho = \\rho_0 \\exp \\left[ c_p \\left(p - p_0\\right) \\right]

        The reference density and the compressibility are taken from the material
        constants of the reference component, while the reference pressure is accessible
        by mixin; a typical implementation will provide this in a variable class.

        Parameters:
            phase: The single fluid phase.

        Returns:
            A function representing above expression on some domains.

        """

        def rho(domains: pp.SubdomainsOrBoundaries) -> pp.ad.Operator:
            rho_ref = Scalar(
                self.fluid.reference_component.density, "reference_fluid_density"
            )
            rho_ = rho_ref * self.pressure_expo

We note a couple of things:

1. The method `density_of_phase` takes a `phase` instance and is supposed to return a callable. The callable is implemented locally as `def rho(...)` and the function itself is returned.
2. This constitutive law makes no usage of the argument `phase`. If it is used in a multiphase setup, every phase will get `rho` assigned as it's density function.
3. `rho` uses only the reference density of the `fluid.reference_component`. Other components would have no effect on the density.

With above points it is trivial to conclude, that this particular constitutive law is meant for single-phase, single-component fluids.

There are a couple of such function factories which are defined by the [FluidMixin](../src/porepy/compositional/compositional_mixins.py#1042). All of them can be overwritten with some custom constitutive law.
We mention the most important ones here:

- [density_of_phase](../src/porepy/compositional/compositional_mixins.py#1346)
- [specific_enthalpy_of_phase](../src/porepy/compositional/compositional_mixins.py#1392)
- [viscosity_of_phase](../src/porepy/compositional/compositional_mixins.py#1411)
- [thermal_conductivity_of_phase](../src/porepy/compositional/compositional_mixins.py#1430)

By default, they will return a `no_property_function`, which will raise an error if ever called. This will support the user in the modeling process, drawing attention to the fact that some constitutive law is missing.

Again, by virtue of the mixin framework, constitutive laws must be on top of the MRO in order to override the default implementation.

### Equation of State approach

The equation of state approach is designed for thermodynamic models, which are either too costly to be implemented via AD, or use some external library to compute values and derivative values.

The user must define two things to utilize this approach:

1. Appropriate types of [EquationOfState](../src/porepy/compositional/base.py#340)
2. The formal dependencies of the fluid properties (in above example, $p$ would be a dependency of $\rho$).

Equations of State are designed to operate solely with numerical values. The single method which must be implemented for the evaluation of fluid properties is `compute_phase_properties`:

In [8]:
print(inspect.getsource(pp.compositional.EquationOfState.compute_phase_properties))

    def compute_phase_properties(
        self, phase_state: PhysicalState, *thermodynamic_input: np.ndarray
    ) -> PhaseProperties:
        """Method to compute the properties of a phase based any
        thermodynamic input and a given physical state.

        The base class method raises an :obj:`NotImplementedError`.

        Examples:
            1. For a single component mixture, the thermodynamic input may consist of
               just pressure and temperature.
               For isothermal models, it may be just pressure.
            2. For general multiphase-multicomponent mixtures, the thermodynamic input
               may consist of pressure, temperature and partial fractions of components
               in a phase.
            3. For correlations which indirectly represent the solution of the fluid
               phase equilibrium problem, the signature might as well be pressure,
               temperature and independent overall fractions, or other primary flow &
     

We observe that this method takes an arbitrary number of `*thermodynamic_input`.
This input represents the formal dependencies (with above example, a single numpy array would be fed to this method containing pressure values per cell).

These formal dependencies can be defined by the user and the framework will do the rest.

In [9]:
from typing import Callable
import numpy as np


class MyVerySpecialFluid(MySpecialFluid):
    pressure: Callable[[pp.SubdomainsOrBoundaries], pp.ad.Variable]
    temperature: Callable[[pp.SubdomainsOrBoundaries], pp.ad.Variable]

    def dependencies_of_phase_properties(
        self, phase: pp.Phase
    ) -> Sequence[Callable[[pp.GridLikeSequence], pp.ad.Variable]]:
        aqu, non_aqu, gas = self.fluid.phases

        if phase in [aqu, non_aqu]:
            return [self.pressure]
        elif phase == gas:
            return [self.pressure, self.temperature]
        else:
            raise ValueError

    def get_phase_configuration(
        self, components: Sequence[pp.compositional.ComponentLike]
    ) -> Sequence[
        tuple[pp.compositional.EquationOfState, pp.compositional.PhysicalState, str]
    ]:
        h2o, co2 = components

        eos_aqu = AqueousEoS([h2o])
        eos_nonaqu = NonAqueousEoS([co2])
        eos_gas = GasEoS(components)
        # We can configure phases by also passing an Equation of State.
        # It does not have to be an EoS per phase. Phases can also share the EoS object.
        # The return type of changes and we return the EoS as the first tuple element.
        return [
            (eos_aqu, pp.compositional.PhysicalState.liquid, "aqueous"),
            (eos_nonaqu, pp.compositional.PhysicalState.liquid, "non_aqueous"),
            (eos_gas, pp.compositional.PhysicalState.gas, "gaseous"),
        ]


class AqueousEoS(pp.compositional.EquationOfState):
    def __init__(self, components):
        super().__init__(components)
        # Store relevant data from the expected components.

    # NOTE: The signature of this method takes the physical state since an EoS can in
    # general be used for both liquid and gas-like phases.
    def compute_phase_properties(self, phase_state, *thermodynamic_input):
        assert phase_state == pp.compositional.PhysicalState.liquid
        assert len(thermodynamic_input) == 1
        p = thermodynamic_input[0]
        # Proceed with computations of properties using stored component data.
        return return_dummy_properties(phase_state, self._nc, *thermodynamic_input)


class NonAqueousEoS(pp.compositional.EquationOfState):
    def __init__(self, components):
        super().__init__(components)
        # Store relevant data from the expected components.

    def compute_phase_properties(self, phase_state, *thermodynamic_input):
        assert phase_state == pp.compositional.PhysicalState.liquid
        assert len(thermodynamic_input) == 1
        p = thermodynamic_input[0]
        # Proceed with computations of properties using stored component data.
        return return_dummy_properties(phase_state, self._nc, *thermodynamic_input)


class GasEoS(pp.compositional.EquationOfState):
    def __init__(self, components):
        super().__init__(components)
        # Store relevant data from the expected components.

    def compute_phase_properties(self, phase_state, *thermodynamic_input):
        assert phase_state == pp.compositional.PhysicalState.gas
        assert len(thermodynamic_input) == 2
        p, T = thermodynamic_input
        # Proceed with computations of properties using stored component data.
        return return_dummy_properties(phase_state, self._nc, *thermodynamic_input)


def return_dummy_properties(
    state: pp.compositional.PhysicalState, num_components: int, *input: np.ndarray
) -> pp.compositional.PhaseProperties:
    num_dependencies = len(input)
    num_cells = input[0].shape[0]

    return pp.compositional.PhaseProperties(
        h=np.zeros(num_cells),
        rho=np.zeros(num_cells),
        state=state,
        phis=np.zeros((num_components, num_cells)),
        dh=np.zeros((num_dependencies, num_cells)),
        drho=np.zeros((num_dependencies, num_cells)),
        dphis=np.zeros((num_components, num_dependencies, num_cells)),
        mu=np.zeros(num_cells),
        dmu=np.zeros((num_dependencies, num_cells)),
        kappa=np.zeros(num_cells),
        dkappa=np.zeros((num_dependencies, num_cells)),
    )

Using `MyVerySpecialFluid`, any phase property $\psi$ of the aqueous and non-aqueous phase will have a signature

$$
\psi_a = \psi_a(p)
~,~
\psi_{na} = \psi_{na}(p),
$$

and analogously for the gas phase it will be $\psi_g = \psi_g(p, T)$.

By providing respective Equations of state, `*thermodynamic_input` will be a tuple containing an array of pressure values, or two arrays of pressure and temperature values respectively.

Under the hood, the `FluidMixin` will detect that formal dependencies of phase properties are defined.
If so, every phase property will be instantiated as a [SurrogateFactory](../src/porepy/numerics/ad/surrogate_operator.py#360).
The values and derivative values of the respective [SurrogateOperators](../src/porepy/numerics/ad/surrogate_operator.py#141) will be populated with the results of the respective Equation of state.

> **Note on formal dependencies:**
>
> The dependencies declared by `dependencies_of_phase_properties` must be variables in the mathematical and AD sense.
> Such quantities have identity blocks in their `.jac` attributes when parsed. The indices of these identity blocks are used by the framework to insert derivative values returned by the EoS into the global Jacobian.
> This would not work with general AD operators/ non-variables, since they have multiple entries per row in `.jac`.

### Hybrid approach

It is also possible to mix the two approaches, using an EoS for some properties, and using heuristic laws for others.

Use cases include situations where some quantities are not readily available from an EoS, or can be simplified without introducing a large modelling error. Notably, viscosity and thermal conductivity are such quantities, which in many EoS are treated as secondary state functions in the thermodynamic sense.

Let's define a hypothetical constitutive law for viscosity for `MyVerySpecialFluid`:

- constant viscosity of the aqueous phase using some reference value of the water component.
- analogously for the non-aqueous phase using CO2.
- some fictional mixing rule for the gas phase, using the partial fractions.

In [10]:
class MyViscosity(pp.PorePyModel):
    def viscosity_of_phase(
        self, phase: pp.Phase
    ) -> pp.compositional.compositional_mixins.ExtendedDomainFunctionType:
        aqu, non_aqu, gas = self.fluid.phases
        h2o, co2 = self.fluid.components

        mu: Callable[[pp.SubdomainsOrBoundaries], pp.ad.Operator]

        if phase == aqu:

            def mu(domains: pp.SubdomainsOrBoundaries) -> pp.ad.Operator:
                return pp.ad.Scalar(h2o.viscosity)

        elif phase == non_aqu:

            def mu(domains: pp.SubdomainsOrBoundaries) -> pp.ad.Operator:
                return pp.ad.Scalar(co2.viscosity)

        elif phase == gas:

            def mu(domains: pp.SubdomainsOrBoundaries) -> pp.ad.Operator:
                mu_h2o = pp.ad.Scalar(h2o.viscosity)
                mu_co2 = pp.ad.Scalar(co2.viscosity)

                return (
                    gas.partial_fraction_of[h2o](domains) * mu_h2o
                    + gas.partial_fraction_of[co2](domains) * mu_co2
                )

        return mu

**Important:**
When using the hybrid approach, constitutive laws which are supposed to overwrite the
behavior of the EoS must **strictly** be above the customized fluid (again, keyword mixins).

We now have everything to define a flow & transport model with our complex fluid.
With some desired geometry, IC and BC values, we can define a model as follows:

In [11]:
class MyCompositionalFlowModel(
    MyViscosity,  # MyViscosity above MyVerySpecialFluid
    MyVerySpecialFluid,
    pp.compositional_flow.CompositionalFlowTemplate,
):
    pass

model = MyCompositionalFlowModel()

# Creates the fluid.
model.prepare_simulation()

print('number of components:', model.fluid.num_components)
print('number of phases:', model.fluid.num_phases)

for phase in model.fluid.phases:
    print('---')
    print(f"{phase.name} phase:")
    print('components in phase:', [component.name for component in phase])
    print('type of phase properties:', type(phase.density))

number of components: 2
number of phases: 3
---
aqueous phase:
components in phase: ['H2O']
type of phase properties: <class 'porepy.numerics.ad.surrogate_operator.SurrogateFactory'>
---
non_aqueous phase:
components in phase: ['CO2']
type of phase properties: <class 'porepy.numerics.ad.surrogate_operator.SurrogateFactory'>
---
gaseous phase:
components in phase: ['H2O', 'CO2']
type of phase properties: <class 'porepy.numerics.ad.surrogate_operator.SurrogateFactory'>


Let's assert that the degrees of freedom are as we expect.

In [12]:
sds = model.mdg.subdomains()
print(
    "Feed fraction is variable:",
    [
        (comp.name, isinstance(comp.fraction(sds), pp.ad.Variable))
        for comp in model.fluid.components
    ],
)
print(
    "Saturation is variable:",
    [
        (phase.name, isinstance(phase.saturation(sds), pp.ad.Variable))
        for phase in model.fluid.phases
    ],
)

for phase in model.fluid.phases:
    print("---")
    print(f"{phase.name} phase:")
    print(
        "Partial fraction is variable:",
        [
            (
                comp.name,
                isinstance(phase.partial_fraction_of[comp](sds), pp.ad.Variable),
            )
            for comp in phase
        ],
    )

Feed fraction is variable: [('H2O', False), ('CO2', True)]
Saturation is variable: [('aqueous', False), ('non_aqueous', True), ('gaseous', True)]
---
aqueous phase:
Partial fraction is variable: [('H2O', False)]
---
non_aqueous phase:
Partial fraction is variable: [('CO2', False)]
---
gaseous phase:
Partial fraction is variable: [('H2O', False), ('CO2', True)]


Since we have no equilibrium type defined, phase fractions are not introduced.
Calling them will raise an error:

In [13]:
try:
    model.fluid.reference_phase.fraction(sds)
except pp.compositional.CompositionalModellingError as err:
    print(err)

Phase fractions are not defined in model without equilibrium. A re-formulation using saturations is required.


Since phase fractions are not defined, the specific enthalpy of the fluid is also not defined since it is implemented
as a sum of phase enthalpies weighed with phase fractions:

In [14]:
try:
    model.fluid.specific_enthalpy(sds)
except pp.compositional.CompositionalModellingError as err:
    print(err)

Phase fractions are not defined in model without equilibrium. A re-formulation using saturations is required.


The `CompositionalFlowTemplate` does not suffer from above error, because it uses an independent fluid enthalpy variable.
Hence `model.fluid.specific_enthalpy` is never called.

For completeness, we mention that an alternative [Fluid](../src/porepy/compositional/base.py#681) can be implemented and
used in [create_fluid](../src/porepy/compositional/compositional_mixins.py#1091), which does not rely on phase fractions at all.
Though this is out of the scope of this tutorial.

Finally, let's see if the viscosities are as we expect, and not trivial as returned by the dummy EoS.

In [15]:
aqu, non_aqu, gas = model.fluid.phases

for phase in model.fluid.phases:
    print('---')
    print(f"{phase.name} phase:")
    mu = phase.viscosity(sds)
    mu_val = model.equation_system.evaluate(mu)
    print('Type and value of viscosity:', type(mu), mu_val)

---
aqueous phase:
Type and value of viscosity: <class 'porepy.numerics.ad.operators.Scalar'> 0.001
---
non_aqueous phase:
Type and value of viscosity: <class 'porepy.numerics.ad.operators.Scalar'> 0.0001
---
gaseous phase:
Type and value of viscosity: <class 'porepy.numerics.ad.operators.Operator'> [0.001 0.001 0.001 0.001]


Since we have trivial IC values, the fraction of CO2 in the gas phase is zero, and the fraction of water in the gas phase is by unity equal to 1. Therefore, the viscosity of the gas phase is equal the viscosity of the pure water phase.

## Final remarks

`MyCompositionalFlowModel` is not runable due to the system not being closed! As we have seen, some dangling variables are left, namely 2 saturation variables and 1 partial fraction variable. Since the `CompositionalFlowTemplate` covers only the PDEs, we require some closure in the form of interpolation and local elimination of the dangling variables, or some equilibrium system.

But as far as this tutorial goes, we have explored all relevant aspects of fluid modeling in PorePy.
